In [13]:
from model.pdf import Pdf
import pdfplumber
from pprint import pprint

In [57]:
with pdfplumber.open('CDMATR~1.PDF') as pdf:
    text = pdf.pages[0]

    table = text.extract_table(table_settings={"vertical_strategy": "text", 
                                         "horizontal_strategy": "text", 
                                         "snap_tolerance": 1,})
    
    pprint(table, width=250)
    

[['MEDIDOR N° 15159341', '', '', '', '', '', '', '', '', ''],
 ['CONST. POTENCIA 0,84000', '', '', '', '', '', '', '', '', ''],
 ['CONST. ATIVO    0,21000', '', '', '', '', '', '', '', '', ''],
 ['CONST. REATIVO  0,21000', '', '', '', '', '', '', '', '', ''],
 ['CONSUMO PONTA VM', '8.718.104', '8.872.295', '32.380,1', '', '', '', '', '', ''],
 ['CONSUMO FORA PONTA CAPACIT. VM', '19.349.389', '19.652.495', '63.652,3', '', '', '', '', '', ''],
 ['CONSUMO FORA PONTA INDUTIVO VM', '70.852.410', '71.939.251', '228.236,6', '', '', '', '', '', ''],
 ['DEMANDA PONTA', '822', '760', '638,4', '', '', '', '', '', ''],
 ['DEMANDA FORA PONTA CAPACITIVA', '553', '526', '441,8', '', '', '', '', '', ''],
 ['DEMANDA FORA PONTA INDUTIVA', '882', '845', '709,8', '', '', '', '', '', ''],
 ['ENERGIA REATIVA PONTA', '2.630.692', '2.664.229', '7.042,8', '', '', '', '', '', ''],
 ['ENERGIA REATIVA F. PONTA IND.', '25.517.765', '25.812.977', '61.994,5', '', '', '', '', '', ''],
 ['UFER PONTA', '25.423', '25.42

In [6]:
with pdfplumber.open('CDMATR~1.PDF') as pdf:
	texts = list(filter(lambda i: i, (map(lambda it: j.split('\n') if (j := it.extract_text()) else None, pdf.pages))))

pdf = Pdf()

In [7]:
numbers = texts[0][2].split(' ')
pdf.n_instalacao, pdf.n_cliente, pdf.n_medidor = numbers[0], numbers[1], numbers[2]
pdf.cnpj = texts[0][3].split('CPF/CNPJ:')[1].split(' ')[1]
pdf.endereco = texts[0][6]

grava = False

for text in texts[0]:
	if 'CEP' in text:
		grava = True

	if grava:
		pdf.cep = text.split(' ')[1]

	if ' - ' in text:
		grava = False
		

In [11]:
linhasTabela = []
for text in texts[0]:
	if  'CCI' in text:
		grava = True
		
	if 'Valor dos Tributos:' in text:
		grava = False

	if grava:
		linhasTabela.append(text)
		

pdf.tabela = linhasTabela

pprint(pdf.to_string(), width=150)


{'cep': '05280-000',
 'cnpj': '07.158.073/0001-55',
 'endereco': 'EST DE LIGACAO 248 ',
 'n_cliente': '10011949',
 'n_instalacao': '200116756',
 'n_medidor': '15159341',
 'tabela': ['CCI DESCRIÇÃO ALNETITEURRIOAR  LAETITUUARLA  kRWE/GkIWSThR/kAvDaOrh   FAkTWURhA/kDvOar khW/ TARIFA C/ ICMS  BASE ICMS IACLMÍQS  ICMS '
            'VALOR ITMAPROIFSAT OS/S ',
            'MEDIDOR N° 15159341',
            'CONST. POTENCIA 0,84000',
            'CONST. ATIVO    0,21000',
            'CONST. REATIVO  0,21000',
            'CONSUMO PONTA VM 8.718.104 8.872.295 32.380,1',
            'CONSUMO FORA PONTA CAPACIT. VM 19.349.389 19.652.495 63.652,3',
            'CONSUMO FORA PONTA INDUTIVO VM 70.852.410 71.939.251 228.236,6',
            'DEMANDA PONTA 822 760 638,4',
            'DEMANDA FORA PONTA CAPACITIVA 553 526 441,8',
            'DEMANDA FORA PONTA INDUTIVA 882 845 709,8',
            'ENERGIA REATIVA PONTA 2.630.692 2.664.229 7.042,8',
            'ENERGIA REATIVA F. PONTA IND. 25.517.